In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, SpatialDropout1D, SimpleRNN
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D, RNN
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

In [2]:
test = pd.read_csv('maybe_final_data.csv')

In [51]:
test.drop(columns=['Unnamed: 0'], inplace=True)
test.head()

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,stay,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,stay,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,stay,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,stay,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
test.tweet[:5]

0    assuming acceleration of to but in a comfortab...
1    is capable of transporting satellite to orbit ...
2                                                  yup
3                                                 part
4    fly to most place on earth in under min and an...
Name: tweet, dtype: object

In [10]:
test.tweet = test.tweet.astype(str)

In [24]:
max_fatures = 3000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['tweet'].values)
X = tokenizer.texts_to_sequences(test['tweet'].values)
X = pad_sequences(X)

In [40]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 28, 128)           384000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 639,391
Trainable params: 639,391
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
y = pd.get_dummies(test['signal_x']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1761, 28) (1761, 3)
(441, 28) (441, 3)


In [42]:
model.fit(X_train, y_train, epochs=20, validation_split=0.1, batch_size=32, verbose=2)

Train on 1584 samples, validate on 177 samples
Epoch 1/20
 - 3s - loss: 1.0883 - acc: 0.3946 - val_loss: 1.0911 - val_acc: 0.3842
Epoch 2/20
 - 2s - loss: 1.0644 - acc: 0.4249 - val_loss: 1.0815 - val_acc: 0.3898
Epoch 3/20
 - 2s - loss: 0.9799 - acc: 0.5417 - val_loss: 1.0748 - val_acc: 0.4859
Epoch 4/20
 - 2s - loss: 0.8039 - acc: 0.6679 - val_loss: 1.1540 - val_acc: 0.4294
Epoch 5/20
 - 2s - loss: 0.6370 - acc: 0.7487 - val_loss: 1.2890 - val_acc: 0.4463
Epoch 6/20
 - 2s - loss: 0.5000 - acc: 0.8043 - val_loss: 1.5953 - val_acc: 0.4350
Epoch 7/20
 - 2s - loss: 0.3966 - acc: 0.8472 - val_loss: 1.6626 - val_acc: 0.4181
Epoch 8/20
 - 2s - loss: 0.3237 - acc: 0.8794 - val_loss: 1.8014 - val_acc: 0.4068
Epoch 9/20
 - 2s - loss: 0.3109 - acc: 0.8807 - val_loss: 2.0110 - val_acc: 0.3785
Epoch 10/20
 - 2s - loss: 0.2364 - acc: 0.9066 - val_loss: 1.9642 - val_acc: 0.4237
Epoch 11/20
 - 2s - loss: 0.2060 - acc: 0.9129 - val_loss: 2.3607 - val_acc: 0.4181
Epoch 12/20
 - 2s - loss: 0.2094 - acc

In [49]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 3.30
acc: 0.32


## further testing below

In [52]:
test.head()

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,stay,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,stay,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,stay,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,stay,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
XX = test.drop(columns=['signal_x', 'tweet'])

In [55]:
XX

(2202, 3856)

In [80]:
# embed_dim = 128
# lstm_out = 196

# model = Sequential()
# model.add(Embedding(3857, embed_dim, input_length = XX.shape[1]))
# model.add(SpatialDropout1D(0.4))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(12, activation='tanh'))
# model.add(Dense(3, activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())

In [98]:
# create and fit the SimpleRNN model
model2 = Sequential()
model2.add(Dense(1929, input_dim=3856, kernel_initializer='normal', activation='tanh'))
model2.add(Dense(200, activation='tanh'))
model2.add(Dense(70, activation='tanh'))
model2.add(Dense(10, activation='tanh'))
model2.add(Dense(3, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 1929)              7440153   
_________________________________________________________________
dense_24 (Dense)             (None, 200)               386000    
_________________________________________________________________
dense_25 (Dense)             (None, 70)                14070     
_________________________________________________________________
dense_26 (Dense)             (None, 10)                710       
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 33        
Total params: 7,840,966
Trainable params: 7,840,966
Non-trainable params: 0
_________________________________________________________________
None


In [97]:
# create and fit the SimpleRNN model
model3 = Sequential()
model3.add(Dense(1929, input_dim=3856, kernel_initializer='normal', activation='tanh'))
model3.add(Dense(100, activation='tanh'))
model3.add(Dense(10, activation='tanh'))
model3.add(Dense(3, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 1929)              7440153   
_________________________________________________________________
dense_20 (Dense)             (None, 100)               193000    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 33        
Total params: 7,634,196
Trainable params: 7,634,196
Non-trainable params: 0
_________________________________________________________________
None


In [101]:
model2.fit(X_train, y_train, epochs=20, validation_split=.1, batch_size=44)

Train on 1584 samples, validate on 177 samples
Epoch 1/20
1584/1584 [==============================] - 7s 5ms/step - loss: 1.1257 - acc: 0.3876 - val_loss: 1.0996 - val_acc: 0.3842
Epoch 2/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0921 - acc: 0.4059 - val_loss: 1.0934 - val_acc: 0.3842
Epoch 3/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0900 - acc: 0.4059 - val_loss: 1.0933 - val_acc: 0.3842
Epoch 4/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0883 - acc: 0.4059 - val_loss: 1.0931 - val_acc: 0.3842
Epoch 5/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0869 - acc: 0.4059 - val_loss: 1.1011 - val_acc: 0.3842
Epoch 6/20
1584/1584 [==============================] - 4s 2ms/step - loss: 1.0871 - acc: 0.4059 - val_loss: 1.0938 - val_acc: 0.3842
Epoch 7/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0880 - acc: 0.4059 - val_loss: 1.0942 - val_acc: 0.3842
Epoch 8/20
1584

In [102]:
model3.fit(X_train, y_train, epochs=20, validation_split=.1, batch_size=44)

Train on 1584 samples, validate on 177 samples
Epoch 1/20
1584/1584 [==============================] - 5s 3ms/step - loss: 1.1570 - acc: 0.3756 - val_loss: 1.1105 - val_acc: 0.3842
Epoch 2/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0921 - acc: 0.4053 - val_loss: 1.1004 - val_acc: 0.3842
Epoch 3/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0879 - acc: 0.4059 - val_loss: 1.1026 - val_acc: 0.3842
Epoch 4/20
1584/1584 [==============================] - 4s 2ms/step - loss: 1.0872 - acc: 0.3971 - val_loss: 1.1033 - val_acc: 0.3842
Epoch 5/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0884 - acc: 0.4059 - val_loss: 1.0968 - val_acc: 0.3842
Epoch 6/20
1584/1584 [==============================] - 3s 2ms/step - loss: 1.0867 - acc: 0.4059 - val_loss: 1.1028 - val_acc: 0.3842
Epoch 7/20
1584/1584 [==============================] - 4s 2ms/step - loss: 1.0853 - acc: 0.4040 - val_loss: 1.1034 - val_acc: 0.3842
Epoch 8/20
1584

In [100]:
yy = pd.get_dummies(test['signal_x']).values
X_train, X_test, y_train, y_test = train_test_split(XX.values, yy, test_size = 0.20, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1761, 3856) (1761, 3)
(441, 3856) (441, 3)


In [91]:
model.fit(X_train, y_train, epochs=20, validation_split=0.1, batch_size=32, verbose=2)

In [ ]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [106]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3856, 128)         493696    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 3856, 128)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_7 (Dense)              (None, 12)                2364      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 39        
Total params: 750,899
Trainable params: 750,899
Non-trainable params: 0
_________________________________________________________________
